In [1]:
import pandas as pd

In [ ]:
# Logic : All files under app_running follows similar structure.
# Strategy : Reading a sample file and using it to generalize the rest 

df_00 = pd.read_csv('dataset/app_usage/running_app_u00.csv')
df_00.head()

,id,device,timestamp,RUNNING_TASKS_baseActivity_mClass,RUNNING_TASKS_baseActivity_mPackage,RUNNING_TASKS_id,RUNNING_TASKS_numActivities,RUNNING_TASKS_numRunning,RUNNING_TASKS_topActivity_mClass,RUNNING_TASKS_topActivity_mPackage
0,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.google.android.gm.ConversationListActivity...,com.google.android.gm,256,1,1,com.google.android.gm.ConversationListActivity...,com.google.android.gm
1,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.android.launcher2.Launcher,com.android.launcher,2,1,1,com.android.launcher2.Launcher,com.android.launcher
2,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.rhmsoft.fm.FileManager,com.rhmsoft.fm,265,1,1,com.rhmsoft.fm.FileManager,com.rhmsoft.fm
3,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.android.systemui.recent.RecentsActivity,com.android.systemui,260,1,1,com.android.systemui.recent.RecentsActivity,com.android.systemui
4,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.google.android.apps.paco.ExperimentManager...,com.google.android.apps.paco,262,1,1,com.google.android.apps.paco.ExperimentManager...,com.google.android.apps.paco


In [4]:
df_00.columns

Index(['id', 'device', 'timestamp', 'RUNNING_TASKS_baseActivity_mClass',
       'RUNNING_TASKS_baseActivity_mPackage', 'RUNNING_TASKS_id',
       'RUNNING_TASKS_numActivities', 'RUNNING_TASKS_numRunning',
       'RUNNING_TASKS_topActivity_mClass',
       'RUNNING_TASKS_topActivity_mPackage'],
      dtype='object')

In [31]:
df_00.nunique()

id                                     5233
device                                    1
timestamp                              5233
RUNNING_TASKS_baseActivity_mClass       129
RUNNING_TASKS_baseActivity_mPackage      92
RUNNING_TASKS_id                        267
RUNNING_TASKS_numActivities               8
RUNNING_TASKS_numRunning                  8
RUNNING_TASKS_topActivity_mClass        190
RUNNING_TASKS_topActivity_mPackage       92
dtype: int64

In [10]:
df_00.shape

(59879, 10)

In [7]:
cols_to_drop = ['id', 'RUNNING_TASKS_id', 'RUNNING_TASKS_baseActivity_mClass', 'RUNNING_TASKS_baseActivity_mPackage', 'RUNNING_TASKS_numActivities' , 'RUNNING_TASKS_topActivity_mClass']
cleaned_df = df_00.drop(columns=cols_to_drop)

In [ ]:
FINAL_APPS = [
 'com.google.android.gm',
 'com.google.android.email',
 'com.android.chrome',
 'com.google.android.youtube',
 'com.spotify.mobile.android.ui',
 'com.google.android.music',
 'com.snapchat.android',
 'com.google.android.talk',
 'com.skype.raider',
 'com.google.android.calendar',
 'com.dropbox.android',
 'com.android.contacts',
 'com.google.android.gallery3d',
 'com.google.android.googlequicksearchbox',
 'com.politico.android',
 'com.guardian',
 'com.google.earth',
 'com.nprpodcastplayer.app',
 'com.piazza.android',
 'com.MoodPanda',
 'org.bewellapp'
]
cleaned_df = cleaned_df[cleaned_df['RUNNING_TASKS_topActivity_mPackage'].isin(FINAL_APPS)]

In [ ]:
cleaned_df.loc[:,'date-time'] = pd.to_datetime(cleaned_df['timestamp'],unit='s')
cleaned_df.loc[:,'date'] = cleaned_df['date-time'].dt.date

In [54]:
cleaned_df.head()

,device,timestamp,RUNNING_TASKS_numRunning,RUNNING_TASKS_topActivity_mPackage,date,app_switch,date-time
20,1977b545-a88f-4903-a7ae-2c434de4be49,1364095883,3,com.google.android.gallery3d,2013-03-24,1,2013-03-24 03:31:23
16,1977b545-a88f-4903-a7ae-2c434de4be49,1364095883,1,com.android.chrome,2013-03-24,1,2013-03-24 03:31:23
14,1977b545-a88f-4903-a7ae-2c434de4be49,1364095883,1,org.bewellapp,2013-03-24,1,2013-03-24 03:31:23
25,1977b545-a88f-4903-a7ae-2c434de4be49,1364095883,1,com.android.contacts,2013-03-24,1,2013-03-24 03:31:23
17,1977b545-a88f-4903-a7ae-2c434de4be49,1364095883,1,com.google.android.gm,2013-03-24,1,2013-03-24 03:31:23


In [20]:
cleaned_df = cleaned_df.sort_values(by='timestamp')
cleaned_df['app_switch'] = (
    cleaned_df['RUNNING_TASKS_topActivity_mPackage'] != cleaned_df['RUNNING_TASKS_topActivity_mPackage'].shift(1)
).astype(int)

In [29]:
cleaned_df = cleaned_df[cleaned_df['app_switch']!=0]

In [50]:
final_app_df = cleaned_df.groupby(by='date').agg(
    total_app_switch= ('app_switch','sum'),
    total_num_tasks = ('RUNNING_TASKS_numRunning','sum'),
    most_used_app = ('RUNNING_TASKS_topActivity_mPackage',
                     lambda x: x.value_counts().idxmax()
                     )
)
final_app_df

,total_app_switch,total_num_tasks,most_used_app
date,,,
2013-03-24,92,96,com.google.android.gm
2013-03-25,137,126,com.android.chrome
2013-03-26,212,212,com.google.android.gm
2013-03-27,201,208,com.android.contacts
2013-03-28,301,297,com.android.chrome
...,...,...,...
2013-06-11,469,421,com.google.android.gallery3d
2013-06-12,351,359,com.google.android.gm
2013-06-13,61,74,com.google.android.gm


In [ ]:
device_id = df_00['device'].unique()[0]
final_app_df['device'] = device_id

Final Function to sort running apps data

In [23]:
def get_app_running_df(filepath):
    cols_to_drop = [
        'id',
        'RUNNING_TASKS_id',
        'RUNNING_TASKS_baseActivity_mClass',
        'RUNNING_TASKS_baseActivity_mPackage',
        'RUNNING_TASKS_numActivities',
        'RUNNING_TASKS_topActivity_mClass'
    ]
    FINAL_APPS = [
        'com.google.android.gm',
        'com.google.android.email',
        'com.android.chrome',
        'com.google.android.youtube',
        'com.spotify.mobile.android.ui',
        'com.google.android.music',
        'com.snapchat.android',
        'com.google.android.talk',
        'com.skype.raider',
        'com.google.android.calendar',
        'com.dropbox.android',
        'com.android.contacts',
        'com.google.android.gallery3d',
        'com.google.android.googlequicksearchbox',
        'com.politico.android',
        'com.guardian',
        'com.google.earth',
        'com.nprpodcastplayer.app',
        'com.piazza.android',
        'com.MoodPanda',
        'org.bewellapp'
    ]
    
    app_running_df = pd.read_csv(filepath)
    cleaned_df = app_running_df.drop(columns=cols_to_drop)
    
    cleaned_df = cleaned_df[cleaned_df['RUNNING_TASKS_topActivity_mPackage'].isin(FINAL_APPS)]
    
    cleaned_df.loc[:,'datetime'] = pd.to_datetime(cleaned_df['timestamp'],unit='s')
    cleaned_df.loc[:,'date'] = cleaned_df['datetime'].dt.date
    
    cleaned_df = cleaned_df.sort_values(by='timestamp')
    cleaned_df['app_switch'] = (
        cleaned_df['RUNNING_TASKS_topActivity_mPackage'] != cleaned_df['RUNNING_TASKS_topActivity_mPackage'].shift(1)
    ).astype(int)
    
    cleaned_df = cleaned_df[cleaned_df['app_switch']!=0]
    
    final_app_df = cleaned_df.groupby(by='date').agg(
    total_app_switch= ('app_switch','sum'),
    total_num_tasks = ('RUNNING_TASKS_numRunning','sum'),
    most_used_app = ('RUNNING_TASKS_topActivity_mPackage',
                     lambda x: x.value_counts().idxmax()
                     )
    )
    device_id = app_running_df['device'].iloc[0]
    
    final_app_df['device'] = device_id
    final_app_df['uid'] = filepath.split('_')[-1].replace('.csv','')
    
    return final_app_df

In [ ]:
# Merging all student datas
import os
root_dir = '../dataset/app_usage'
# os.path.join('dataset','app_usage')
user_files = os.listdir(path=root_dir)
combined_dfs = []
for file in user_files:
    print('Reading : ',file)
    file_path = os.path.join(root_dir,file)
    df= get_app_running_df(file_path)
    combined_dfs.append(df)
app_running_df = pd.concat(combined_dfs)

,total_app_switch,total_num_tasks,most_used_app,device,uid
date,,,,,
2013-03-24,100,100,com.google.android.gm,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-25,123,123,com.android.chrome,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-26,145,145,com.android.chrome,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-27,308,347,org.bewellapp,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-28,131,131,org.bewellapp,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-29,260,344,org.bewellapp,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-30,390,390,org.bewellapp,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-03-31,404,404,org.bewellapp,83147b17-0754-41b9-a03a-b5a490c60372,u01
2013-04-01,398,398,org.bewellapp,83147b17-0754-41b9-a03a-b5a490c60372,u01
